# Fractional Differencing

Summary of exercises 5.1, 5.2 and proposed by the author. The other exercises will be treated as "projects" in the omonim folder :)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import sys

sys.path.append('../../scripts')
from fracdiff import *

Generate an IID gaussian time series (memory-less stationary series):
- ADF test
- ADF test on cumulative sum of observations
- Differenciate the cumulatve sum twice and AFD test

In [2]:
#for reproducibility
np.random.seed(42)
#number of obs 
T = 1000
# generate series
X = np.random.normal(0,1,T)
# cumualtive sum
Y = np.cumsum(X)
# second order differentiation
Z = np.diff(Y, n=2)

# ADF test on X
result_X = adfuller(X)
print(f'ADF Statistic for X: {round(result_X[0],4)} with p-value: {round(result_X[1],4)}')
# ADF test on Y
result_Y = adfuller(Y)
print(f'ADF Statistic for Y: {round(result_Y[0],4)} with p-value: {round(result_Y[1],4)}')
# ADF test on X
result_Z = adfuller(Z)
print(f'ADF Statistic for Z: {round(result_Z[0],4)} with p-value: {round(result_Z[1],4)}')
#       

ADF Statistic for X: -31.8111 with p-value: 0.0
ADF Statistic for Y: -0.966 with p-value: 0.7655
ADF Statistic for Z: -11.5366 with p-value: 0.0


Generate a timeseries that follows a sinusoidal function (stationary series with memory)
- ADF test
- Shift all observation by a constant and compute the cumulative sum
    - ADF test
    - Apply an expanding window fracdiff with tau = 1e-2, what is the minimum d to get a p-val below 5%
    - Apply an FFD with tau = 1e-5, what is the minimum d to get a p-val below 5%

In [3]:
#for reproducibility
np.random.seed(42)
#number of obs 
T = 1000
# sinusoidal series
amplitude = 10
frequency = 0.02
X = amplitude * np.sin(frequency * np.arange(T)) + np.random.normal(0,1,T)
# cumualtive sum
c = 2
Y = np.cumsum(X+c)

# ADF test on X
result_X = adfuller(X)
print(f'ADF Statistic for X: {round(result_X[0],4)} with p-value: {round(result_X[1],4)}')
# ADF test on Y
result_Y = adfuller(Y)
print(f'ADF Statistic for Y: {round(result_Y[0],4)} with p-value: {round(result_Y[1],4)}')
# ADF test on X


ADF Statistic for X: -3.7441 with p-value: 0.0035
ADF Statistic for Y: -1.2929 with p-value: 0.6324


In [ ]:
# apply an expanding window fracdiff with tau = 1e-2, what is the minimum d to get a p-val below 5%?

d = 0.0
d_max = 2
step = 0.01
tau = 1e-2
Y = pd.DataFrame(Y, columns=['Y']) 

while d <= d_max:
    Y_fd = fracDiff(Y, d=d, threshold=tau)
    result_Y_fd = adfuller(Y_fd.dropna())
    print(f'ADF Statistic for FFD Y with d={round(d,2)}: {round(result_Y_fd[0],4)} with p-value: {round(result_Y_fd[1],4)}')
    if result_Y_fd[1] < 0.05:
        print(f'Minimum d to get a p-val below 5% is {round(d,2)}')
        break
    d += step

ADF Statistic for FFD Y with d=0.0: -1.2382 with p-value: 0.6569
ADF Statistic for FFD Y with d=0.01: -1.8262 with p-value: 0.3675
ADF Statistic for FFD Y with d=0.02: -3.004 with p-value: 0.0345
Minimum d to get a p-val below 5% is 0.02


In [15]:
d = 0.0
d_max = 2
step = 0.01
tau = 1e-5
Y = pd.DataFrame(Y, columns=['Y']) 

while d <= d_max:
    Y_fd = fracDiff_FFD(Y, d=d, thres=tau)
    Y_fd_col = Y_fd['Y'].dropna()
    
    #skip if the result of afd is empty
    if len(Y_fd_col) == 0:
        # skip to next d
        d += step
        continue
    
    result_Y_fd = adfuller(Y_fd_col)
    print(f'ADF Statistic for FFD Y with d={round(d,2)}: {round(result_Y_fd[0],4)} with p-value: {round(result_Y_fd[1],4)}')
    
    if result_Y_fd[1] < 0.05:
        print(f'Minimum d to get a p-val below 5% is {round(d,2)}')
        break
    
    d += step


ADF Statistic for FFD Y with d=0.0: -1.2929 with p-value: 0.6324
ADF Statistic for FFD Y with d=0.01: 0.7233 with p-value: 0.9903
ADF Statistic for FFD Y with d=0.49: 0.7628 with p-value: 0.991
ADF Statistic for FFD Y with d=0.5: 1.2216 with p-value: 0.9961
ADF Statistic for FFD Y with d=0.51: 1.6191 with p-value: 0.9979
ADF Statistic for FFD Y with d=0.52: 0.052 with p-value: 0.9626
ADF Statistic for FFD Y with d=0.53: -0.9683 with p-value: 0.7646
ADF Statistic for FFD Y with d=0.54: -2.494 with p-value: 0.1169
ADF Statistic for FFD Y with d=0.55: -2.1003 with p-value: 0.2444
ADF Statistic for FFD Y with d=0.56: -2.9349 with p-value: 0.0414
Minimum d to get a p-val below 5% is 0.56
